In [ ]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import glob
import os
import matplotlib.pyplot as plt

In [ ]:
save_path = "/home/aevans/nwp_bias/src/machine_learning/data/profiler_images/"

In [ ]:
radiometer_data_path = (
    "/home/aevans/nysm/archive/profiler/netcdf/proc-range/2024/02/20240201.nc"
)

In [ ]:
df = xr.open_mfdataset(f"{radiometer_data_path}")

In [ ]:
df

In [ ]:
df = df.to_dataframe()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# sites = df['station'].unique()
# for i in np.arange(2018,2026):
#     for s in sites:
#         os.mkdir(f'{save_path}/{i}/{s}')

In [ ]:
df["temperature"] = df["temperature"] - 273.13
df["ir_temperature"] = df["ir_temperature"] - 273.13
df["dewpoint"] = df["dewpoint"] - 273.13
df["relative_humidity"] = df["relative_humidity"] / 100

In [ ]:
for c in df.columns:
    print("column", c)
    # Count NaN values in the 'temperature' column
    nan_count = df[c].isna().sum()

    percent = (nan_count / 423504) * 100
    print("percent of column that is corrupted", percent)

In [ ]:
# Drop columns with the suffix '_qc'
df = df.drop(columns=df.filter(like="_qc").columns)
df = df.drop(columns=df.filter(like="surface").columns)
drop_list = ["v", "w", "velocity", "direction", "u", "cnr", "rws"]
df = df.drop(columns=drop_list)
df = df[df["range"] <= 5000]
df.head()

In [ ]:
df = df[df["station"] == "PROF_ALBA"]
df.fillna(-999, inplace=True)
df = df.drop(columns="station")
df["time"] = pd.to_datetime(df["time"])

In [ ]:
# Extract unique days
unique_days = df["time"].dt.date.unique()

In [ ]:
unique_days

In [ ]:
# # Filter rows where the hour matches the query time's hour
# filtered_df = df[df['datetime'].dt.hour == query_time.hour]

In [ ]:
df

In [ ]:
skip_list = ["time", "range"]
for c in df.columns:
    if c in skip_list:
        continue
    else:
        var_pivot = df.pivot(index="range", columns="time", values=c)
        var_array = var_pivot.to_numpy()
        try:
            stacked_array = np.vstack((stacked_array, var_array))
        except:
            stacked_array = var_array

In [ ]:
stacked_array